In [36]:
!pip install -q llama-index-llms-gemini

In [37]:
!pip install -q llama-index google-generativeai

In [38]:
!pip install -q llama-index-embeddings-gemini

In [39]:
!pip install -q chromadb

In [40]:
!pip install -q llama-index-vector-stores-chroma

In [1]:
!pip install -q llama-index-llms-anthropic

In [1]:
import os
import chromadb

os.environ["ANTHROPIC_API_KEY"] = ""
os.environ["GOOGLE_API_KEY"] = ""

In [2]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.gemini import Gemini
from llama_index.llms.anthropic import Anthropic
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import VectorIndexRetriever


/home/vishal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

chroma_client = chromadb.PersistentClient(path="./main")
chroma_collection = chroma_client.get_or_create_collection("astchroma")
documents = SimpleDirectoryReader('data').load_data() 

In [5]:
from llama_index.core.node_parser import CodeSplitter

In [6]:
import tree_sitter_go as tsgo
from tree_sitter import Language, Parser
# Define the Go language and parser
GO_LANGUAGE = Language(tsgo.language())
parser = Parser(GO_LANGUAGE)
parser.language = GO_LANGUAGE 

In [7]:
splitter = CodeSplitter.from_defaults(language='go',parser=parser)



In [8]:
from llama_index.core import PromptTemplate

text_qa_template_str = ( 
       """
You are a code refactoring assistant specialised in golang. Your task is to review the provided code and suggest using appropriate functions from the language's standard library or popular packages to make the code more concise and efficient, while maintaining its functionality.\n
Do not include any reasoning, comments or text in the output except the code.\n
Do not include a main function. \n
Kindly use package main BEFORE the imported packages. \n

Here is an example for your reference in which the manual parsing and validation of JSON is made concise using json.Unmarshal() function from encoding/json library while maintaining functionality and semantics:  \n

Example Input:
package main

import (
    "fmt"
    "errors"
    "strings"
)

func parseAndValidateJSON(jsonStr string) (map[string]string, error) {{
    jsonStr = removeWhitespace(jsonStr)

    if !startsWith(jsonStr, "{{") || !endsWith(jsonStr, "}}") {{
        return nil, errors.New("invalid JSON format")
    }}

    jsonStr = removeBraces(jsonStr)

    pairs := splitIntoPairs(jsonStr)
    result := make(map[string]string)

    for _, pair := range pairs {{
        kv := splitKeyValue(pair)
        if len(kv) != 2 {{
            return nil, errors.New("invalid key-value pair")
        }}

        key := removeQuotes(kv[0])
        value := removeQuotes(kv[1])

        if key == "" {{
            return nil, errors.New("invalid key")
        }}

        result[key] = value
    }}

    return result, nil
}}

// Helper functions (simulated std library functions)
func removeWhitespace(str string) string {{
    return strings.ReplaceAll(str, " ", "")
}}

func startsWith(str, prefix string) bool {{
    return strings.HasPrefix(str, prefix)
}}

func endsWith(str, suffix string) bool {{
    return strings.HasSuffix(str, suffix)
}}

func removeBraces(str string) string {{
    return str[1 : len(str)-1]
}}

func splitIntoPairs(str string) []string {{
    return strings.Split(str, ",")
}}

func splitKeyValue(str string) []string {{
    return strings.Split(str, ":")
}}

func removeQuotes(str string) string {{
    return strings.Trim(str, "\"")
}}


Example Output:

package main

import (
    "fmt"
    "encoding/json"
)

func parseAndValidateJSON(jsonStr string) (map[string]string, error) {{
    var data map[string]string
    if err := json.Unmarshal([]byte(jsonStr), &data); err != nil {{
        return nil, errors.New("invalid JSON format")
    }}
    return data, nil
}}

input code snippet: \n
{query_str}



"""
     )
text_qa_template = PromptTemplate(text_qa_template_str)

In [9]:
import extract_relevant_part
import code_to_ast

In [10]:
with open ("go_code_in.go", "r") as f_in:
    go_code_in = f_in.read()

go_codes_out = extract_relevant_part.get_relevant_part(go_code_in, parser)
for go_code_out in go_codes_out:
    print(go_code_out)
    print("\n\n")


['package main\n', 'import (\n\t"fmt"\n\t"strings"\n)\n', 'func subtract(a, b int) int {\n\treturn a - b\n}']



['package main\n', 'import (\n\t"fmt"\n\t"strings"\n)\n', 'func multiply(a, b int) int {\n\treturn a * b\n}']



['package main\n', 'import (\n\t"fmt"\n\t"strings"\n)\n', 'func main() {\n\t// Arithmetic functions\n\tfmt.Println("Add 10 + 5 =", add(10, 5))\n\tfmt.Println("Subtract 10 - 5 =", subtract(10, 5))\n\tfmt.Println("Multiply 10 * 5 =", multiply(10, 5))\n\n\t// Sum array\n\tnums := []int{1, 2, 3, 4, 5}\n\tfmt.Println("Sum of array =", sumArray(nums))\n\n\t// String functions\n\ts := "hello world"\n\tfmt.Println("Capitalized:", capitalize(s))\n\tfmt.Println("Reversed:", reverse(s))\n\tfmt.Println("Capitalized and Reversed:", capitalizeAndReverse(s))\n\n\t// Struct and method\n\tp := Person{FirstName: "John", LastName: "Doe", Age: 30}\n\tfmt.Println("Full Name:", p.FullName())\n\tfmt.Println(p.Introduce())\n}']



['package main\n', 'import (\n\t"fmt"\n\t"strings"\n)\n',

In [10]:
tokenizer = Anthropic().tokenizer

In [11]:
llm = Anthropic(model="claude-3-haiku-20240307")
embed_model = GeminiEmbedding(model_name="models/text-embedding-004")


In [12]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
    transformations=[splitter]
)

In [13]:
query_engine = index.as_query_engine(text_qa_template=text_qa_template,llm=llm)

In [14]:
for go_code_out in go_codes_out:
    query_str = "".join(go_code_out)
    response=query_engine.query(query_str)
    print(response.response)
    print("\n\n\n")


package main

import (
    "fmt"
)

type Person struct {
    FirstName string
    LastName  string
    Age       int
}

func (p Person) FullName() string {
    return fmt.Sprintf("%s %s", p.FirstName, p.LastName)
}

func (p Person) Introduce() string {
    return fmt.Sprintf("Hello, my name is %s and I am %d years old.", p.FullName(), p.Age)
}




package main

import (
    "fmt"
    "strings"
)

func capitalizeAndReverse(s string) string {
    return reverse(strings.Title(s))
}

func reverse(s string) string {
    runes := []rune(s)
    for i, j := 0, len(runes)-1; i < j; i, j = i+1, j-1 {
        runes[i], runes[j] = runes[j], runes[i]
    }
    return string(runes)
}




package main

import (
    "fmt"
    "strings"
)

func add(a, b int) int {
    return a + b
}

func subtract(a, b int) int {
    return a - b
}

func multiply(a, b int) int {
    return a * b
}

func sumArray(nums []int) int {
    var sum int
    for _, num := range nums {
        sum += num
    }
    return sum
}

